In [ ]:
import numpy as np
import pandas as pd
import math
import sys
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
class Config:
    __PopulationSize = 50 # Population Size
    __MaxDomain = 500 # variable upper limit
    __MinDomain = -500 # variable lower limit
    __Lambda = 1.5 # parameter for Levy flight
    __Pa = 0.25
    __Step_Size = 0.01
    __Dimension = 3 # The number of dimension
    __Trial = 1
    __Iteration = 3000

    @classmethod
    def get_population_size(cls):
        return cls.__PopulationSize

    @classmethod
    def get_Pa(cls):
        return cls.__Pa

    @classmethod
    def get_iteration(cls):
        return cls.__Iteration

    @classmethod
    def get_trial(cls):
        return cls.__Trial

    @classmethod
    def get_dimension(cls):
        return cls.__Dimension

    @classmethod
    def get_max_domain(cls):
        return cls.__MaxDomain

    @classmethod
    def set_max_domain(cls, _max_domain):
        cls.__MaxDomain = _max_domain

    @classmethod
    def get_min_domain(cls):
        return cls.__MinDomain

    @classmethod
    def set_min_domain(cls, _min_domain):
        cls.__MinDomain = _min_domain

    @classmethod
    def get_lambda(cls):
        return cls.__Lambda

    @classmethod
    def set_lambda(cls, _lambda):
        cls.__Lambda = _lambda

    @classmethod
    def get_stepsize(cls):
        return cls.__Step_Size


In [ ]:
train = pd.read_csv("train_data.csv")
def calculation(weights):
    fitness = rmse(weights)
    return fitness

def rmse(weights):
    predicted = []
    actual = []
    n = len(weights)
    for row in train.iterrows():
        x = row[1]['INPUT1']*weights[n-3] + row[1]['INPUT2']*weights[n-2] + row[1]['INPUT3']*weights[n-1]
        predicted.append(x)
        actual.append(row[1]['OUTPUT'])
    error = mean_squared_error(actual,predicted)
    return math.sqrt(error)

In [ ]:
def levy_flight(Lambda):
    #generate step from levy distribution
    sigma1 = np.power((math.gamma(1 + Lambda) * np.sin((np.pi * Lambda) / 2)) \
                      / math.gamma((1 + Lambda) / 2) * np.power(2, (Lambda - 1) / 2), 1 / Lambda)
    sigma2 = 1
    u = np.random.normal(0, sigma1, size=Config.get_dimension())
    v = np.random.normal(0, sigma2, size=Config.get_dimension())
    step = u / np.power(np.fabs(v), 1 / Lambda)

    return step


class Individual:
    def __init__(self):
        self.__position = np.random.rand(Config.get_dimension()) * (Config.get_max_domain() - Config.get_min_domain())  + Config.get_min_domain()
        self.__fitness = calculation(self.__position) 

    def get_position(self):
        return self.__position

    def set_position(self, position):
        self.__position = position

    def get_fitness(self):
        return self.__fitness

    def set_fitness(self, fitness):
        self.__fitness = fitness

    def abandon(self):
        # abandon some variables
        for i in range(len(self.__position)):
            p = np.random.rand()
            if p < Config.get_Pa():
                self.__position[i] = np.random.rand() * (Config.get_max_domain() - Config.get_min_domain())  + Config.get_min_domain()

    def get_cuckoo(self):

        step_size = Config.get_stepsize() * levy_flight(Config.get_lambda())

        # Update position
        self.__position = self.__position + step_size

        # Simple Boundary Rule
        for i in range(len(self.__position)):
            if self.__position[i] > Config.get_max_domain():
                self.__position[i] = Config.get_max_domain()
            if self.__position[i] < Config.get_min_domain():
                self.__position[i] = Config.get_min_domain()

    def print_info(self,i):
        print("id:","{0:3d}".format(i),
              "|| fitness:",str(self.__fitness).rjust(14," "),
              "|| position:",np.round(self.__position,decimals=4))



In [ ]:
#cuckoo-search
for trial in range(Config.get_trial()):
    np.random.seed(trial)

    results_list = [] # fitness list
    iteration_list = []
    cs_list = []
    """Generate Initial Population"""
    for p in range(Config.get_population_size()):
        cs_list.append(Individual())

    """Sort List"""
    cs_list = sorted(cs_list, key=lambda ID: ID.get_fitness())

    """Find Initial Best"""
    BestPosition = cs_list[0].get_position()
    BestFitness = calculation(cs_list[0].get_position())

    """↓↓↓Main Loop↓↓↓"""
    for iteration in range(Config.get_iteration()):

        """Generate New Solutions"""
        for i in range(len(cs_list)):
            cs_list[i].get_cuckoo()
            cs_list[i].set_fitness(calculation(cs_list[i].get_position()))

            """random choice (say j)"""
            j = np.random.randint(low=0, high=Config.get_population_size())
            while j == i: #random id[say j] ≠ i
                j = np.random.randint(0, Config.get_population_size())

            # for minimize problem
            if(cs_list[i].get_fitness() < cs_list[j].get_fitness()):
                cs_list[j].set_position(cs_list[i].get_position())
                cs_list[j].set_fitness(cs_list[i].get_fitness())

        """Sort (to Keep Best)"""
        cs_list = sorted(cs_list, key=lambda ID: ID.get_fitness())

        """Abandon Solutions (exclude the best)"""
        for a in range(1,len(cs_list)):
            r = np.random.rand()
            if(r < Config.get_Pa()):
                cs_list[a].abandon()
                cs_list[a].set_fitness(calculation(cs_list[a].get_position()))

        """Sort to Find the Best"""
        cs_list = sorted(cs_list, key=lambda ID: ID.get_fitness())

        if cs_list[0].get_fitness() < BestFitness:
            BestFitness = cs_list[0].get_fitness()
            BestPosition = cs_list[0].get_position()

        results_list.append(BestFitness)
        iteration_list.append(iteration)

    plt.plot(results_list,iteration_list)
    plt.ylabel('Fitness')
    plt.xlabel('Iteration')
    plt.grid() 
    plt.show()

